# Projet 6 - Feature image

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os
from os import listdir
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, auc, roc_auc_score, roc_curve
from glob import glob

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalAveragePooling1D, Flatten, Dense, Dropout 
from tensorflow.keras.layers import Rescaling, RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing
import time
from PIL import Image


# os.environ["TF_KERAS"]='1'
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [30]:
data = pd.read_csv('data/Flipkart/flipkart_com-ecommerce_sample_1050.csv')
df = data.copy()
df.sample(5)

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
120,16c18ce0affca13159e362227c83683c,2016-03-25 17:27:56 +0000,http://www.flipkart.com/quilt-india-floral-cus...,Quilt India Floral Cushions Cover,"[""Home Furnishing >> Cushions, Pillows & Cover...",CPCEH2BTEYRFXHP5,1000.0,500.0,16c18ce0affca13159e362227c83683c.jpg,False,Key Features of Quilt India Floral Cushions Co...,No rating available,No rating available,Quilt India,"{""product_specification""=>[{""key""=>""Brand"", ""v..."
420,6c3768639e82ceaeab681e083b897672,2015-12-01 06:13:00 +0000,http://www.flipkart.com/d-link-dsl-2750u-wirel...,D-Link DSL-2750U Wireless N 300 ADSL2+ 4-Port ...,"[""Computers >> Network Components >> Routers >...",RTRD4KY4UNGYCZFG,3140.0,2069.0,6c3768639e82ceaeab681e083b897672.jpg,False,Buy D-Link DSL-2750U Wireless N 300 ADSL2+ 4-P...,4.3,4.3,D-Link,"{""product_specification""=>[{""key""=>""Firewall"",..."
183,3830dda3491cd5b28ed42ae544bc44f0,2015-12-01 12:40:44 +0000,http://www.flipkart.com/rockmantra-blue-water-...,Rockmantra Blue Water Abstracts Ceramic Mug,"[""Kitchen & Dining >> Coffee Mugs >> Rockmantr...",MUGEAGGKDCJHRYBY,499.0,199.0,3830dda3491cd5b28ed42ae544bc44f0.jpg,False,Rockmantra Blue Water Abstracts Ceramic Mug (5...,No rating available,No rating available,NaN,"{""product_specification""=>[{""key""=>""Type"", ""va..."
144,3847353a4b8de1efc504d589aea74ffe,2015-12-01 12:40:44 +0000,http://www.flipkart.com/rockmantra-tele-phoner...,Rockmantra Tele PhoneRMMW50000160 Ceramic Mug,"[""Kitchen & Dining >> Coffee Mugs >> Rockmantr...",MUGEAGGKUQE2HYPZ,499.0,199.0,3847353a4b8de1efc504d589aea74ffe.jpg,False,Rockmantra Tele PhoneRMMW50000160 Ceramic Mug ...,No rating available,No rating available,NaN,"{""product_specification""=>[{""key""=>""Type"", ""va..."
498,11502f02601ff063bf7db0a618354652,2016-01-07 05:50:25 +0000,http://www.flipkart.com/times-times-51-party-w...,Times Times_51 Party-Wedding Analog Watch - F...,"[""Watches >> Wrist Watches >> Times Wrist Watc...",WATE54YZWYXR6Z7J,599.0,599.0,11502f02601ff063bf7db0a618354652.jpg,False,Times Times_51 Party-Wedding Analog Watch - F...,No rating available,No rating available,NaN,"{""product_specification""=>[{""key""=>""Chronograp..."


In [37]:
print(df['uniq_id'].nunique())
print(df['image'].nunique())

1050
1050


Il y a autant de valeurs uniques dans la colonne image que dans la colonne uniq_id. Cela signifie que l'ensemble des valeurs de la colonne image est unique et que donc on peut metrte les valeurs de la colonne image en index.

In [38]:
df = pd.read_csv("data/Flipkart/flipkart_com-ecommerce_sample_1050.csv",
                 sep=',', 
                 index_col = 'image',
                 encoding ='utf-8')

df_pict = df[['product_name', 'description']].copy('deep')
df_pict.sample(3)

,product_name,description
image,,
d36e6f157d1155369f763a0a434fb129.jpg,Digisol DG-BG4100N,Buy Digisol DG-BG4100N only for Rs. 1470 from ...
1d1be744e491ed61e705c20e4a72320d.jpg,Oxyglow Lacto Bleach & Fruit Massage Cream Wit...,Oxyglow Lacto Bleach & Fruit Massage Cream Wit...
ad55b7ac6f7284b90d31a1a2f175df8d.jpg,meSleep Abstract Cushions Cover,Buy meSleep Abstract Cushions Cover at Rs. 599...


On importe les images et on les met dans un dictionnaire

In [ ]:
file_path = 'data/Flipkart/Images'

li_files = df_pict.index

## importing pictures
dict_img_orig = {}

# loop on the pictures in the file
for i, ind in enumerate(li_files,1):
    print('\r', f"{i}/{len(list(li_files))}", end="")
    # cast each image to a np.array
    img = np.array(Image.open(os.path.join(file_path, ind)))

    # put in the dictionnary
    dict_img_orig[ind] = img


 1050/1050
Total time: 8.11 seconds


In [46]:
ser_img_orig = pd.Series(dict_img_orig,
                         index = df_pict.index,
                         name='img_orig')
df_pict = pd.concat([df_pict,
                     ser_img_orig.to_frame()], axis=1)

In [47]:
ser_img_orig

image
55b85ea15a1536d46b7190ad6fff8ce7.jpg    [[[192, 184, 173], [192, 184, 173], [193, 185,...
7b72c92c2f6c40268628ec5f14c6d590.jpg    [[[255, 255, 255], [255, 255, 255], [255, 255,...
64d5d4a258243731dc7bbb1eef49ad74.jpg    [[[255, 255, 255], [255, 255, 255], [255, 255,...
d4684dcdc759dd9cdf41504698d737d8.jpg    [[[255, 255, 255], [255, 255, 255], [255, 255,...
6325b6870c54cd47be6ebfbffa620ec7.jpg    [[[99, 99, 91], [80, 80, 72], [67, 67, 57], [6...
                                                              ...                        
958f54f4c46b53c8a0a9b8167d9140bc.jpg    [[[253, 253, 251], [255, 255, 253], [255, 255,...
fd6cbcc22efb6b761bd564c28928483c.jpg    [[[253, 216, 224], [253, 216, 224], [253, 216,...
5912e037d12774bb73a2048f35a00009.jpg    [[[255, 255, 255], [255, 255, 255], [255, 255,...
c3edc504d1b4f0ba6224fa53a43a7ad6.jpg    [[[52, 52, 54], [54, 54, 56], [51, 51, 53], [5...
f2f027ad6a6df617c9f125173da71e44.jpg    [[[241, 224, 232], [241, 224, 232], [241, 224,...
Name

In [48]:
df_pict

,product_name,description,img_orig
image,,,
55b85ea15a1536d46b7190ad6fff8ce7.jpg,Elegance Polyester Multicolor Abstract Eyelet ...,Key Features of Elegance Polyester Multicolor ...,"[[[192, 184, 173], [192, 184, 173], [193, 185,..."
7b72c92c2f6c40268628ec5f14c6d590.jpg,Sathiyas Cotton Bath Towel,Specifications of Sathiyas Cotton Bath Towel (...,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
64d5d4a258243731dc7bbb1eef49ad74.jpg,Eurospa Cotton Terry Face Towel Set,Key Features of Eurospa Cotton Terry Face Towe...,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
d4684dcdc759dd9cdf41504698d737d8.jpg,SANTOSH ROYAL FASHION Cotton Printed King size...,Key Features of SANTOSH ROYAL FASHION Cotton P...,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
6325b6870c54cd47be6ebfbffa620ec7.jpg,Jaipur Print Cotton Floral King sized Double B...,Key Features of Jaipur Print Cotton Floral Kin...,"[[[99, 99, 91], [80, 80, 72], [67, 67, 57], [6..."
...,...,...,...
958f54f4c46b53c8a0a9b8167d9140bc.jpg,Oren Empower Extra Large Self Adhesive Sticker,Oren Empower Extra Large Self Adhesive Sticker...,"[[[253, 253, 251], [255, 255, 253], [255, 255,..."
fd6cbcc22efb6b761bd564c28928483c.jpg,Wallmantra Large Vinyl Sticker Sticker,Wallmantra Large Vinyl Sticker Sticker (Pack o...,"[[[253, 216, 224], [253, 216, 224], [253, 216,..."
5912e037d12774bb73a2048f35a00009.jpg,Uberlyfe Extra Large Pigmented Polyvinyl Films...,Buy Uberlyfe Extra Large Pigmented Polyvinyl F...,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."


# Plan du projet

## Travail sur l'image

- resizing
- affichage des images

## Algorithme SIFT / ORB / SUR

- Faire les algos
- Visualiser les features avec TSNE

## CNN - Transfer learning

- visualisation des features avec le TSNE